In [ ]:
import pandas as pd
from datetime import datetime, timedelta

import zipfile
from shapely.geometry import Point

import pyproj
import math

from collections import defaultdict
import json

In [149]:
df = pd.read_excel('Report_JR14_Vehicle History Report - data.xlsx',skiprows=10)

In [150]:
'''
    Create Trips from the GPS Data
    1. 
'''
# df.columns

'\n    Create Trips from the GPS Data\n    1. \n'

In [151]:
depot = [-27.426770, 153.150956]
proj_wgs84 = pyproj.Proj(proj='latlong', datum='WGS84')

def convert_to_gtfs_time(ts,st_ts):
    ts_dt = datetime.strptime(ts, "%Y-%m-%dT%H:%M:%S.%f")
    start_trip_dt = pd.to_datetime(st_ts)
    
    ts_dt_obj = ts_dt.date()
    start_trip_dt_obj = start_trip_dt.date()

    if ts_dt_obj == start_trip_dt_obj:
        return ((ts_dt.hour * 60) + ts_dt.minute + ts_dt.second)
    elif (ts_dt - start_trip_dt).days > 0:
        return (((ts_dt - start_trip_dt).seconds) // 60 + ((ts_dt - start_trip_dt).days * 24 * 60 * 60) // 60)
    else:
        print("Error in Timestamp")
        return None

def check_buffer(point1,point2):
    # Convert Shapely Point objects to (longitude, latitude) tuples
    lon1, lat1 = point1.x, point1.y
    lon2, lat2 = point2.x, point2.y

    # Convert latitude and longitude from degrees to radians
    lat1_rad, lon1_rad = math.radians(lat1), math.radians(lon1)
    lat2_rad, lon2_rad = math.radians(lat2), math.radians(lon2)

    # Calculate the distance using pyproj's Geod class
    geod = pyproj.Geod(ellps='WGS84')
    azimuth1, azimuth2, distance = geod.inv(lon1_rad, lat1_rad, lon2_rad, lat2_rad)
    return distance

In [152]:
# session_starts = []

current_session_start = None
on_odometer = None
off_odometer = None

trip_ends = defaultdict(list)
calendar = defaultdict(list)
stops = defaultdict(list) # IMPORTANT
stop_times = defaultdict(list) # IMPORTANT
trips = defaultdict(list) # IMPORTANT
routes = defaultdict(list) # IMPORTANT
shapes = defaultdict(list) 
feed_info = defaultdict(list)
agency = defaultdict(list)

blocks_for_input = {"blocks":[]}

df = df[df['Reason'].isin(['Arr. Way Point','Break End','Break Start','Dep. Way Point','Logoff','Logon','Status Report','Ignition Off','Ignition On','Odometer Offset'])]

agency["agency_id"].append("MGL")
agency["agency_name"].append("Microgrid Labs")
agency["agency_url"].append("https://www.microgridlabs.com")
agency["agency_timezone"].append("Asia/Kolkata")

calendar["start_date"].append(pd.to_datetime(df['GPS'].min()).strftime('%Y%m%d'))
calendar["end_date"].append(pd.to_datetime(df['GPS'].max()).strftime('%Y%m%d'))
calendar["monday"].append("1")
calendar["tuesday"].append("1")
calendar["wednesday"].append("1")
calendar["thursday"].append("1")
calendar["friday"].append("1")
calendar["saturday"].append("1")
calendar["sunday"].append("1")
calendar["service_id"].append("S1")

feed_info["feed_publisher_name"].append("Microgrid Labs")
feed_info["feed_publisher_url"].append("https://www.microgridlabs.com")
feed_info["feed_lang"].append("eng")
feed_info["feed_start_date"].append("20230101")
feed_info["feed_end_date"].append("20250101")
feed_info["feed_version"].append("1")
feed_info["feed_url"].append("https://www.microgridlabs.com")

trip_no = 1
block_id = 1
tid = 0
bid = 0
block_changed = True
blocks_data = []
total_block_distance = 0

prev_start , prev_stop = None, None
for index, row in df.iterrows():
    if row['Reason'] == 'Ignition On':
        current_session_start = index
        on_odometer = row['Odometer']
    elif row['Reason'] == 'Ignition Off' and current_session_start is not None:
        off_odometer = row['Odometer']

        if on_odometer is not None and off_odometer is not None:
            # Set another logic for selecting trips since SaaS only takes trips < 1km in distance for REM
            diff = off_odometer - on_odometer
            if diff > 1:

                flag = False
                
                trips_df = df.loc[current_session_start:index].reset_index()
                
                # Set Trips file here
                trips["trip_id"].append(f"T_{trip_no}")
                trips["route_id"].append(f"R_{trip_no}")
                trips["service_id"].append("S1")

                trips["shape_id"].append(f"Sh_{trip_no}")
                trips["trip_short_name"].append(f"Short Name {trip_no}")
                trips["direction_id"].append("1")

                start_point  = Point(trips_df['Latitude'][0],trips_df['Longitude'][0])
                if trip_no > 1 :
                    start_point  = Point(trips_df['Latitude'][0],trips_df['Longitude'][0])
                
                # Set Block Logic here
                # Block logic can also be setup using df.[prev_start and prev_stop] and df.[current_session_start and index]
                if check_buffer(Point(depot[0],depot[1]),start_point) <= 10 and (prev_start is not None and prev_stop is not None) and (prev_stop <= len(df)):
                    curr_t_start = df.iloc[current_session_start]['GPS']
                    prev_t_end = df.iloc[prev_stop]['GPS']
                    
                    if (curr_t_start - prev_t_end) < timedelta(minutes=30) and curr_t_start >= prev_t_end:
                        block_id = block_id
                    else:
                        bid+=1
                        block_id+=1
                        block_changed = True
                        flag = True

                trips["block_id"].append(f"B{block_id}")
                # Set Routes file here
                routes["agency_id"].append("MGL")
                routes["route_id"].append(f"R_{trip_no}")
                routes["route_short_name"].append(f"RSn {trip_no}")
                routes["route_long_name"].append(f"Long Name {trip_no}")
                routes["route_type"].append(3)

                prev_start = current_session_start
                prev_stop = index

                trip_start_day = trips_df['GPS'][0].strftime('%Y%m%d')

                if flag:
                    total_block_distance = 0 # total_block_distance will become zero when ever we add a new block id 
                    print(total_block_distance)
                    
                # To get the start and end time for each trip and to get the start time for each block id
                start_date_time = trips_df['GPS'].iloc[0]
                end_date_time = trips_df['GPS'].iloc[-1]
                start_time = convert_to_gtfs_time(start_date_time.strftime('%Y-%m-%dT%H:%M:%S.%f'), trip_start_day) 
                end_time = convert_to_gtfs_time(end_date_time.strftime('%Y-%m-%dT%H:%M:%S.%f'), trip_start_day)

                # To get start and end latitude and start and end longitude for each trip and to get the start latitude and start longitude for each block id 
                start_lat = trips_df['Latitude'].iloc[0]
                start_lon = trips_df['Longitude'].iloc[0]
                end_lat = trips_df['Latitude'].iloc[-1]
                end_lon = trips_df['Longitude'].iloc[-1]

                # To get the total trip distance
                trips_df['Next_Latitude'] = trips_df['Latitude'].shift(-1)
                trips_df['Next_Longitude'] = trips_df['Longitude'].shift(-1)
                trips_df['Start_Point'] = trips_df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
                trips_df['End_Point'] = trips_df.apply(lambda row: Point(row['Next_Longitude'], row['Next_Latitude']), axis=1)
                trips_df['Distance'] = trips_df.iloc[:-1].apply(lambda row: check_buffer(row['Start_Point'], row['End_Point']), axis=1)
                trips_df.at[trips_df.index[-1], 'Distance'] = 0
                total_trip_distance = trips_df['Distance'].sum()
                # To get the total block distance
                total_block_distance += total_trip_distance 
                print(total_block_distance)

                if block_changed:
                    block = {
                        "id": bid,
                        "blockId": f"B{block_id}",
                        "dh_st_time": int(start_time),  
                        "startTime": int(start_time),  
                        "endTime": 0,  
                        "dh_end_time": 0,  
                        "lon_start": float(start_lon),  
                        "lat_start": float(start_lat),  
                        "distance": total_block_distance,  
                        "vehicleEff": "",  
                        "vehicleModel": "",  
                        "trips": []
                    }

                HVAC_energy = 0.0 
                Aux_energy = 0.0
                stop_portion = 0.5
                payload = 0.5

                trip = {
                    "id": tid,
                    "trip_id": f"T_{trip_no}",
                    "type": 3,
                    "start_time": int(start_time),
                    "start_lat": float(start_lat),
                    "start_lon": float(start_lon),
                    "end_time": int(end_time),
                    "end_lat": float(end_lat),
                    "end_lon": float(end_lon),
                    "distance": float(total_trip_distance),
                    "eff": "",
                    "direction": 1,
                    "stops": [],
                    "route_id": f"R_{trip_no}",
                    "route_name": f"RSn {trip_no}",
                    "shape_id": f"Sh_{trip_no}",
                    "HVAC_energy": float(HVAC_energy),  
                    "Aux_energy": float(Aux_energy),  
                    "stop_portion": float(stop_portion),  
                    "payload": float(payload)  
                }

                sid = 0
                for id,trips_row in trips_df.iterrows():   

                    stop_id = round(abs(float(trips_row['Latitude'])),2).__str__() + round(abs(float(trips_row['Longitude'])),2).__str__() + pd.to_datetime(trips_row['GPS']).strftime('%Y%m%d%H%M%S') + id.__str__() 

                    # Set Stops File - DONE
                    if id in [0, len(trips_df) - 1]:
                        stops["stop_id"].append(f"{stop_id}")
                        stops["stop_name"].append(f"Stop {id}")
                        stops["stop_lat"].append(trips_row['Latitude'])
                        stops["stop_lon"].append(trips_row['Longitude'])

                        # Set Stop Times File - DONE
                        stop_times["stop_id"].append(f"{stop_id}")
                        stop_times["trip_id"].append(f"T_{trip_no}")

                        # Timepoint is required for start_time and end_time
                        stop_times["timepoint"].append("1")
                        if id == 0:
                            stop_times["stop_sequence"].append("1")
                        else:
                            stop_times["stop_sequence"].append("2")

                        stop_times["arrival_time"].append(f"{convert_to_gtfs_time(trips_row['GPS'].strftime('%Y-%m-%dT%H:%M:%S.%f'),trip_start_day)}")
                        stop_times["departure_time"].append(f"{convert_to_gtfs_time(trips_row['GPS'].strftime('%Y-%m-%dT%H:%M:%S.%f'),trip_start_day)}")

                        stop = {
                            "id": sid,
                            "stop_id": f"{stop_id}",
                            "arrival_time": convert_to_gtfs_time(trips_row['GPS'].strftime('%Y-%m-%dT%H:%M:%S.%f'), trip_start_day),
                            "departure_time": convert_to_gtfs_time(trips_row['GPS'].strftime('%Y-%m-%dT%H:%M:%S.%f'), trip_start_day),
                            "lat": float(trips_row['Latitude']),
                            "lon": float(trips_row['Longitude'])
                        }
                        
                        trip["stops"].append(stop)

                        sid += 1
                        
                    # Set Shapes File - DONE
                    shapes["shape_id"].append(f"Sh_{trip_no}")
                    shapes["shape_pt_lat"].append(trips_row['Latitude'])
                    shapes["shape_pt_lon"].append(trips_row['Longitude'])
                    shapes["shape_pt_sequence"].append(f"{int(id)+1}")
                    
                # Proceed to next Trip ID
                trip_no += 1
                tid += 1

                block["trips"].append(trip)

                if block_changed:
                    blocks_data.append(block)
                    
                block_changed = False

        else:
            # Do something when the Ignition On or Off row does not have any odometer value
            print("Ignition On or Off do not have an Odometer Value")
            
json_data = {
    "blocks": blocks_data
}

for block in json_data["blocks"]:
    block["startDepot"] = 1  
    block["endDepot"] = 1  
    block["cords"] = []
    for trip in block["trips"]:
        for stop in trip["stops"]:
            coordinates = [float(stop["lon"]), float(stop["lat"])]
            block["cords"].append(coordinates)

364.2001359864913
458.7991766896197
503.9323569224074
800.3412663308877
897.7175374181365
965.0916414466999
1319.99475782314
1378.1105561337442
1430.1165822853484
1829.6111802591956
0
425.88046371018817
478.8921529941379
566.0230328149783
822.6916217234273
1035.1949951150327
1630.5662316256282
1660.3213723608335
2251.1011740460826
2605.8598290136324
2644.421009339929
2733.095704408689
3073.0714013602746
0
363.11090167091345
416.58019924925406
454.235294497224
744.7537994653885
1169.845074966298
1236.839070555178
1509.3960483701944
1599.5296359474505
1951.361558271602
2318.489235325004
2741.2645418324364
2772.2390897200494
2830.6770076658204
2879.405265758933
3206.7153054606456
3561.5867274804023
3915.7281629510207
4541.657322355801
5191.708411453032
5530.779089693967
0
76.62814095064168
151.3975073401898
512.0600872238924
587.5133652030936
814.0575766179818
993.2567684379778
1308.1594922185116
1365.9004254220292
1707.3940323863446
2107.1417749275824
2168.3050878783993
2198.992770408303

In [153]:
if len(stops["stop_id"]) == len(set(stops["stop_id"])):
    print("Y")
else:
    print(len(stops["stop_id"]))
    print(len(set(stops["stop_id"])))

Y


In [154]:
temp_dir = 'output'
pd.DataFrame(agency).to_csv(f"{temp_dir}/agency.txt",index=False)
pd.DataFrame(feed_info).to_csv(f"{temp_dir}/feed_info.txt",index=False)
pd.DataFrame(calendar).to_csv(f"{temp_dir}/calendar.txt",index=False)
pd.DataFrame(stops).to_csv(f"{temp_dir}/stops.txt",index=False)
pd.DataFrame(stop_times).to_csv(f"{temp_dir}/stop_times.txt",index=False)
pd.DataFrame(trips).to_csv(f"{temp_dir}/trips.txt",index=False)
pd.DataFrame(routes).to_csv(f"{temp_dir}/routes.txt",index=False)
pd.DataFrame(shapes).to_csv(f"{temp_dir}/shapes.txt",index=False)

with open(f"{temp_dir}/output.json", 'w') as json_file:
    json.dump(json_data, json_file, indent=4)

# with zipfile.ZipFile("gtfs_output.zip", "w") as zip_file:
#     for filename in ["agency.txt","feed_info.txt","stop_times.txt", "stops.txt", "trips.txt", "routes.txt","calendar.txt","shapes.txt"]:
#         zip_file.write(f"{temp_dir}/{filename}", arcname=filename)